In [ ]:
import cv2
emotion_dict = ('Angry','contempt' ,'Disgust', 'Fear', 'Happy','Sad', 'Surprise')

device = "cpu"
val_transform = transforms.Compose([
        transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])
net = Deep_Emotion()
net.load_state_dict(torch.load('C:/Users/nourg/Desktop/4th Year/GP/archive/deep_emotion_ck+-100-64-0.005-97%.pt'))
net.to(device)
net.eval()
cap = cv2.VideoCapture(0)

count = 0

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    count +=1
    if True:
        face_cascade = cv2.CascadeClassifier('C:/Users/nourg/anaconda3/Library/etc/haarcascades/haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(frame)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
            resize_frame = cv2.resize(gray[y:y + h, x:x + w], (48, 48))
            X = resize_frame/256
#         cv2.imwrite("roi.jpg", X)
#         plt.imshow(X,'gray')
#         plt.show()
            X = Image.fromarray((X))
            X = val_transform(X).unsqueeze(0)
            with torch.no_grad():
                net.eval()
                out = net(X)
                pred = F.softmax(out,dim=1)
                classs = torch.argmax(pred,1)
                pred = emotion_dict[classs.item()]
            cv2.putText(frame, pred, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
            count = 0
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()